In [1]:
import os
import shutil

import awkward as ak
import dask
import dask_awkward as dak
import mplhep as hep
import uproot
from coffea.dataset_tools import preprocess
from distributed import Client
from matplotlib import pyplot as plt

import egamma_tnp
from egamma_tnp import ElectronTagNProbeFromNTuples
from egamma_tnp.plot import plot_ratio
from egamma_tnp.utils.histogramming import save_hists

/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/egmtnpenv/lib64/python3.9/site-packages/coffea/lookup_tools/evaluator.py:7: FutureWarning: In version 2024.8.0 (target date: 2024-07-31 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.lookup_tools.json_lookup will be removed by August 2024. Please use lumi_tools or correctionlib instead!.
  from coffea.lookup_tools.json_lookup import json_lookup


In [2]:
# Define the ntuples here 


## Caution: 2024C does not have Ele32 WP 
fileset = {
    "events_2024C" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024C.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma1_Run2024C.root": "tnpEleTrig/fitter_tree" 
        },
    },
    "events_2024D" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024D.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma1_Run2024D.root": "tnpEleTrig/fitter_tree" 
        },
    }, 
    "events_2024E" : {
        "files": {
#        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma0_Run2024E.root": "tnpEleTrig/fitter_tree",
#        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-06-04/data_EGamma1_Run2024E.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma0_Run2024Ev1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma0_Run2024Ev2.root": "tnpEleTrig/fitter_tree",   
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma1_Run2024Ev1.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-02/data_EGamma1_Run2024Ev2.root": "tnpEleTrig/fitter_tree",               
        },
    },
    "events_2024F" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-29/data_EGamma0_Run2024F.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/ikrommyd/2024-07-29/data_EGamma1_Run2024F.root": "tnpEleTrig/fitter_tree",    
        },
    },
    "events_2024G" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-26/data_EGamma0_Run2024G.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-26/data_EGamma1_Run2024G.root": "tnpEleTrig/fitter_tree",  
        },
    },
    "events_2024H" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-27/data_EGamma0_Run2024H.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-09-27/data_EGamma1_Run2024H.root": "tnpEleTrig/fitter_tree",  
        },
    }, 
    "events_2024I" : {
        "files": {
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-10-25/data_EGamma0_Run2024I.root": "tnpEleTrig/fitter_tree",
        "/eos/cms/store/group/phys_egamma/tnpTuples/sunayak/2024-10-25/data_EGamma1_Run2024I.root": "tnpEleTrig/fitter_tree",  
        },
    },      

}

fileset_available, fileset_updated = preprocess(fileset, step_size=500_000, skip_bad_files=True)

In [3]:
# Define the HLT paths for wihich the efficiency plots are needed along with the last passing Filter

hlt_paths = {
    "Ele30": "passHLTEle30WPTightGsfTrackIsoFilter",
#    "Ele32": "passHLTEle32WPTightGsfTrackIsoFilter",
#    "Ele115": "passHLTEle115CaloIdVTGsfTrkIdTGsfDphiFilter",
#    "Ele135": "passHLTEle135CaloIdVTGsfTrkIdTGsfDphiFilter",
    "Ele23Ele12Leg1": "passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg1Filter",
    "Ele23Ele12Leg2": "passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg2Filter",
#    "DoubleEle33SeededLeg": "passHLTEle33CaloIdLMWPMS2Filter",
#    "DoubleEle33UnseededLeg": "passHLTDiEle33CaloIdLMWPMS2UnseededFilter",
}

# Check what the purpose of defining the plateau cuts is

plateau_cuts = {
    "Ele30": 35,
    "Ele32": 35,
    "Ele115": 120,
    "Ele135": 140,
    "Ele23Ele12Leg1": 25,
    "Ele23Ele12Leg2": 15,
    "DoubleEle33SeededLeg": 35,
    "DoubleEle33UnseededLeg": 35,
}

In [4]:
def runfilter(events):
    dataset = events.metadata["dataset"]
    print(dataset, "no run cut")
    return events

tnp = ElectronTagNProbeFromNTuples(fileset_available, list(hlt_paths.values()), cutbased_id="passingCutBasedTight122XV1", extra_filter=runfilter)

In [16]:
%%time

to_compute = {}

for name, trigger in hlt_paths.items():
    if name == "Ele115" or name == "Ele135":
        egamma_tnp.binning.set(
            "pt_bins",
            [
                5,
                10,
                15,
                20,
                22,
                26,
                28,
                30,
                32,
                34,
                36,
                38,
                40,
                45,
                50,
                60,
                80,
                100,
                105,
                110,
                115,
                120,
                125,
                130,
                135,
                140,
                145,
                150,
                200,
                250,
                300,
                350,
                400,
            ],
        )
    else:
        egamma_tnp.binning.set(
            "pt_bins",
            [
                5,
                10,
                12,
                14,
                16,
                18,
                20,
                23,
                26,
                28,
                30,
                31,
                32,
                33,
                34,
                35,
                36,
                37,
                38,
                39,
                40,
                41,
                42,
                43,
                44,
                45,
                50,
                60,
                80,
                100,
                150,
                250,
                400,
            ],
        )
    plateau_cut = plateau_cuts[name]
    to_compute[name] = tnp.get_1d_pt_eta_phi_tnp_histograms(
        trigger,
        uproot_options={"allow_read_errors_with_report": True},
        eta_regions_pt={
            "barrel": [0.0, 1.4442],
            "endcap_loweta": [1.566, 2.0],
            "endcap_higheta": [2.0, 2.5],
        },
        plateau_cut=plateau_cut,
    )
    
dak.necessary_columns(to_compute)

events_2024C no run cut
events_2024D no run cut
events_2024E no run cut
events_2024F no run cut
events_2024G no run cut
events_2024H no run cut
events_2024I no run cut
events_2024C no run cut
events_2024D no run cut
events_2024E no run cut
events_2024F no run cut
events_2024G no run cut
events_2024H no run cut
events_2024I no run cut
events_2024C no run cut
events_2024D no run cut
events_2024E no run cut
events_2024F no run cut
events_2024G no run cut
events_2024H no run cut
events_2024I no run cut
CPU times: user 18.3 s, sys: 47.3 ms, total: 18.3 s
Wall time: 18.5 s


{'from-uproot-b6947fc0f4b7952a06b729466db4f66e': frozenset({'el_eta',
            'el_phi',
            'el_pt',
            'el_q',
            'pair_mass',
            'passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg1Filter',
            'passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg2Filter',
            'passHLTEle30WPTightGsfTrackIsoFilter',
            'passingCutBasedTight122XV1',
            'tag_Ele_eta',
            'tag_Ele_pt',
            'tag_Ele_q'}),
 'from-uproot-9d17bfa4a8c24ece6f142f85434a15d1': frozenset({'el_eta',
            'el_phi',
            'el_pt',
            'el_q',
            'pair_mass',
            'passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg1Filter',
            'passHLTEle23Ele12CaloIdLTrackIdLIsoVLTrackIsoLeg2Filter',
            'passHLTEle30WPTightGsfTrackIsoFilter',
            'passingCutBasedTight122XV1',
            'tag_Ele_eta',
            'tag_Ele_pt',
            'tag_Ele_q'}),
 'from-uproot-7b0c4bcd498a55040e091e544d61a480': fro

In [17]:
%%time

(out,) = dask.compute(to_compute)

CPU times: user 2min 25s, sys: 35.2 s, total: 3min
Wall time: 2min 5s


In [18]:
#print(out) 

In [19]:
# Make directory in afs/eos are to store the plots 

for dataset in out["Ele30"][1].keys():
    path = f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{dataset}"
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)

In [20]:
# Save histogram root files and json files in afs directory

for name, res in out.items():
    hists, report = res
    for dataset, report_arr in report.items():
        ak.to_json(
            report_arr,
            f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{dataset}/{name}_report.json",
            num_readability_spaces=1,
            num_indent_spaces=4,
        )
    for dataset, hs in hists.items():
        save_hists(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{dataset}/{name}_hists.root", hs)

In [21]:
hep.style.use("CMS")
hep.style.use(
    {
        "figure.figsize": (6.4, 4.8),
        "font.size": 14,
        "legend.title_fontsize": 14,
        "savefig.bbox": "tight",
    }
)

In [22]:
# Fetch histograms from the root files for plotting

def get_histograms(path):
    with uproot.open(path) as file:
        hpt_barrel_pass = file["pt/barrel/passing"].to_hist()
        hpt_barrel_fail = file["pt/barrel/failing"].to_hist()
        hpt_endcap_loweta_pass = file["pt/endcap_loweta/passing"].to_hist()
        hpt_endcap_loweta_fail = file["pt/endcap_loweta/failing"].to_hist()
        hpt_endcap_higheta_pass = file["pt/endcap_higheta/passing"].to_hist()
        hpt_endcap_higheta_fail = file["pt/endcap_higheta/failing"].to_hist()
        hpt_combined_pass = hpt_barrel_pass + hpt_endcap_loweta_pass + hpt_endcap_higheta_pass
        hpt_combined_fail = hpt_barrel_fail + hpt_endcap_loweta_fail + hpt_endcap_higheta_fail
    
        heta_entire_pass = file["eta/entire/passing"].to_hist()
        heta_entire_fail = file["eta/entire/failing"].to_hist()

        hphi_entire_pass = file["phi/entire/passing"].to_hist()
        hphi_entire_fail = file["phi/entire/failing"].to_hist()

    return (
        hpt_barrel_pass,
        hpt_barrel_fail,
        hpt_endcap_loweta_pass,
        hpt_endcap_loweta_fail,
        hpt_endcap_higheta_pass,
        hpt_endcap_higheta_fail,
        hpt_combined_pass,
        hpt_combined_fail,
        heta_entire_pass,
        heta_entire_fail,
        hphi_entire_pass,
        hphi_entire_fail,
    )

In [23]:
# Plot set-up: legends, tick marks, labelling of x and y-axis

#def pt_low_threshold_plot_setup(**legend_kwargs):
#    plt.xlim(10, 400)
#    plt.ylim(0, 1.2)
#    plt.xlabel(r"Offline electron $P_T$ [GeV]")
#    plt.ylabel(r"Efficiency")
#    plt.xscale("log")
#    plt.xticks([10, 100], [10, 100])
#    plt.xticks(
#        [20, 30, 40, 50, 60, 70, 80, 90, 200, 300, 400],
#        [20, 30, 40, 50, None, None, None, None, 200, 300, 400],
#        minor=True,
#    )
#    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()

# Modified for zoomed version
def pt_low_threshold_plot_setup(**legend_kwargs):
    plt.xlim(20, 50)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $P_T$ [GeV]")
    plt.ylabel(r"Efficiency")
    #plt.xscale("log")
    #plt.xticks([30, 50], [30, 50])
    plt.xticks(
        [20, 25, 30, 35, 40, 45, 50],
        [20, 25, 30, 35, 40, 45, 50],
        minor=True,
    )
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def pt_high_threshold_plot_setup(**legend_kwargs):
    plt.xlim(10, 400)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $P_T$ [GeV]")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def eta_plot_setup(**legend_kwargs):
    plt.xlim(-2.5, 2.5)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $\eta$")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()


def phi_plot_setup(**legend_kwargs):
    plt.xlim(-3.32, 3.32)
    plt.ylim(0, 1.2)
    plt.xlabel(r"Offline electron $\phi$")
    plt.ylabel(r"Efficiency")
    plt.legend(**legend_kwargs) if legend_kwargs else plt.legend()

In [24]:
lumis = {
    "2023D": 9.525,
    "2024C": 7.416594091,
    "2024D": 7.889161918,
    "2024Ev1": 6.279524894,
    "2024Ev2": 5.040346798,
    "2024F": 26.313333799,
    "2024G": 17.604084255,
    "2024H": 999,
    "2024I": 999,
}

In [25]:
# To draw multiple comparisons in single plot, here we have redefined the plot_ratio function and modified it 

from __future__ import annotations

import pathlib

import mplhep as hep
import numpy as np
from matplotlib import pyplot as plt

from egamma_tnp.utils import get_ratio_histogram


def _save_and_close(fig, path, close_figure):
    """Saves a figure at a given location if path is provided and optionally closes it.

    Args:
        fig (matplotlib.figure.Figure): figure to save
        path (Optional[pathlib.Path]): path where figure should be saved, or None to not
            save it
        close_figure (bool): whether to close figure after saving
    """
    if path is not None:
        path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(path)
    if close_figure:
        plt.close(fig)


def plot_efficiency(passing_probes, failing_or_all_probes, denominator_type="failing", **kwargs):
    """Plot the efficiency using the ratio of passing to passing + failing probes.

    Parameters
    ----------
        passing_probes : hist.Hist
            The histogram of the passing probes.
        failing_or_all_probes : hist.Hist
            The histogram of the failing or passing + failing probes.
        denominator_type : str, optional
            The type of the denominator histogram.
            Can be either "failing" or "all".
            The default is "failing".
        **kwargs
            Keyword arguments to pass to hist.Hist.plot1d.

    Returns
    -------
        List[Hist1DArtists]

    """
    ratio_hist, yerr = get_ratio_histogram(passing_probes, failing_or_all_probes, denominator_type)

    return ratio_hist.plot1d(histtype="errorbar", yerr=yerr, xerr=True, flow="none", **kwargs)

def plot_ratio(
    passing_probes1,
    failing_or_all_probes1,
    passing_probes2,
    failing_or_all_probes2,
    label1,
    label2,
    denominator_type="failing",
    *,
    plottype="pt_low_threshold",
    figure_path=None,
    figsize=(6, 6),
    eff1_kwargs=None,
    eff2_kwargs=None,
    effratio_kwargs=None,
    cms_kwargs=None,
    legend_kwargs=None,
    efficiency_label=None,
    ratio_label=None,
    _fig=None,
    _skip_fig1=None
):
    """Plot the ratio of two efficiencies.

    Parameters
    ----------
        passing_probes1 : hist.Hist
            The histogram of the passing probes for the first efficiency.
        failing_or_all_probes1 : hist.Hist
            The histogram of the failing or passing + failing probes for the first efficiency.
        passing_probes2 : hist.Hist
            The histogram of the passing probes for the second efficiency.
        failing_or_all_probes2 : hist.Hist
            The histogram of the failing or passing + failing probes for the second efficiency.
        label1 : str
            The label for the first efficiency.
        label2 : str
            The label for the second efficiency.
        denominator_type : str, optional
            The type of the denominator histogram.
            Can be either "failing" or "all".
            The default is "failing".
        plottype : str, optional
            The type of plot to make. Can be "pt_low_threshold", "pt_high_threshold", "eta", or "phi".
            Defaults is "pt_low_threshold".
        figure_path : str, optional
            The path where the figure should be saved, or None to not save it.
            Defaults is None.
        figsize : tuple of floats or ints, optional
            The size of the figure. Defaults is (6, 6).
        eff1_kwargs : dict, optional
            Keyword arguments to pass to hist.Hist.plot1d for the first efficiency.
        eff2_kwargs : dict, optional
            Keyword arguments to pass to hist.Hist.plot1d for the second efficiency.
        effratio_kwargs : dict, optional
            Keyword arguments to pass to matplotlib.pyplot.errorbar for the ratio.
        cms_kwargs : dict, optional
            Keyword arguments to pass to mplhep.cms.label.
        legend_kwargs : dict, optional
            Keyword arguments to pass to matplotlib.pyplot.legend.
    """
    eff2_default_kwargs = {"color": "k"}
    eff1_default_kwargs = {"color": "r"}
    effratio_default_kwargs = {
        "color": "k",
        "linestyle": "none",
        "marker": ".",
        "markersize": 10.0,
        "elinewidth": 1,
    }
    cms_default_kwargs = {
        "label": "Preliminary",
        "data": True,
        "lumi": "X",
        "year": 2023,
        "com": 13.6,
    }
    legend_default_kwargs = {}

    if eff1_kwargs is None:
        eff1_kwargs = eff1_default_kwargs
    else:
        eff1_kwargs = eff1_default_kwargs | eff1_kwargs

    if eff2_kwargs is None:
        eff2_kwargs = eff2_default_kwargs
    else:
        eff2_kwargs = eff2_default_kwargs | eff2_kwargs

    if effratio_kwargs is None:
        effratio_kwargs = effratio_default_kwargs
    else:
        effratio_kwargs = effratio_default_kwargs | effratio_kwargs

    if cms_kwargs is None:
        cms_kwargs = cms_default_kwargs
    else:
        cms_kwargs = cms_default_kwargs | cms_kwargs

    if legend_kwargs is None:
        legend_kwargs = legend_default_kwargs
    else:
        legend_kwargs = legend_default_kwargs | legend_kwargs

    if efficiency_label is None:
        efficiency_label = "Efficiency"
    if ratio_label is None:
        ratio_label = "Ratio"

    if _fig is None:
        fig = plt.figure(figsize=figsize, layout="constrained")
        gs = fig.add_gridspec(nrows=2, ncols=1, hspace=0, height_ratios=[3, 1])
        ax1 = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1])
    else:
        fig = _fig      
        ax1 = fig.axes[0]
        ax2 = fig.axes[1]

    eff1, efferr1 = get_ratio_histogram(passing_probes1, failing_or_all_probes1, denominator_type)
    eff2, efferr2 = get_ratio_histogram(passing_probes2, failing_or_all_probes2, denominator_type)
   
    plot_efficiency(
        passing_probes1,
        failing_or_all_probes1,
        denominator_type,
        label=label1,
        ax=ax1,
        x_axes_label=None,
        **eff1_kwargs,
     )

    if _skip_fig1 is None:    
        plot_efficiency(
            passing_probes2,
            failing_or_all_probes2,
            denominator_type,
            #label=label2,
            ax=ax1,
            x_axes_label=None,
            **eff2_kwargs,
        )
    centers = passing_probes1.axes.centers[0]
    num = eff1.values()
    denom = eff2.values()
    denom[denom == 0.0] = 1
    with np.errstate(divide="ignore", invalid="ignore"):
        ratio = num / denom
        ratioyerr = np.sqrt((efferr1 / num) ** 2 + (efferr2 / denom) ** 2) * ratio
    ratioxerr = passing_probes1.axes.widths[0] / 2
    ax2.errorbar(centers, ratio, ratioyerr, ratioxerr, **effratio_kwargs)
    ax2.axhline(1, color="k", linestyle="--", linewidth=1)

## THIS LINE MIGHT BE AN ISSUE.
    hep.cms.label(ax=ax1, **cms_kwargs)
    
#    if plottype == "pt_low_threshold":
#        ax1.set_xlim(10, 400)
#        ax2.set_xlim(10, 400)
#        ax2.set_xlabel(r"Offline electron $P_T$ [GeV]")
#        ax1.set_xscale("log")
#        ax2.set_xscale("log")
#        ax2.set_xticks([10, 100], [10, 100])
#        ax2.set_xticks(
#            [20, 30, 40, 50, 60, 70, 80, 90, 200, 300, 400],
#            [20, 30, 40, 50, None, None, None, None, 200, 300, 400],
#            minor=True,
#        )
#        legend_loc = "lower right"
#    # MODIFIED for zoomed pt version
    if plottype == "pt_low_threshold":
        ax1.set_xlim(20, 50)
        ax2.set_xlim(20, 50)
        ax2.set_xlabel(r"Offline electron $P_T$ [GeV]")
        #ax1.set_xscale("log")
        #ax2.set_xscale("log")
        #ax2.set_xticks([30, 50], [30, 50])
        ax2.set_xticks(
            [20, 25, 30, 35, 40, 45, 50],
            [20, 25, 30, 35, 40, 45, 50],
            minor=True,
        )
        legend_loc = "lower right"            
    elif plottype == "pt_high_threshold":
        ax1.set_xlim(10, 400)
        ax2.set_xlim(10, 400)
        ax2.set_xlabel(r"Offline electron $P_T$ [GeV]")
        legend_loc = "lower right"
    elif plottype == "eta":
        ax1.set_xlim(-2.5, 2.5)
        ax2.set_xlim(-2.5, 2.5)
        ax2.set_xlabel(r"Offline electron $\eta$")
        legend_loc = "lower center"
    elif plottype == "phi":
        ax1.set_xlim(-3.32, 3.32)
        ax2.set_xlim(-3.32, 3.32)
        ax2.set_xlabel(r"Offline electron $\phi$")
        legend_loc = "lower center"
    else:
        raise ValueError(f"Invalid plottype {plottype}")

    ax1.set_ylim(0, 1.2)
    #ax2.set_ylim(0.7, 1.3)
    ax2.set_ylim(0.2, 1.8) # Updated
    ax1.set_xlabel(None)
    ax2.set_ylabel(None)
    ax1.set_ylabel(efficiency_label)
    ax2.set_ylabel(ratio_label)
    ax1.set_xticklabels([])
    
    legend = ax1.legend(loc=legend_loc, **legend_kwargs)
    legend.get_title().set_multialignment("center")

    if figure_path is not None:
        figure_path = pathlib.Path(figure_path)
        _save_and_close(fig, figure_path, True)
    return fig

In [36]:
for tala in list(hlt_paths.keys()):
    for data_period in ["events_2024C"]:
        _fig=None
        for mc_dataset in ["events_2024D","events_2024E","events_2024F","events_2024G","events_2024H","events_2024I"]:
            tocompare = [mc_dataset, data_period] # data_period = reference, mc_dataset = target
            run = []
            for folder in tocompare:
                run.append(folder.split("_", 2)[2][3:] if "data" in folder else folder.split("_", 1)[1])
            threshold = tala
            if threshold == "Ele32" or threshold == "Ele30":
                suffix = "WPTight_Gsf"
            elif threshold == "Ele115" or threshold == "Ele135":
                suffix = "CaloIdVT_GsfTrkIdT"
            elif threshold == "DoubleEle33SeededLeg":
                suffix = "CaloIdL_MW\nSeeded leg"
            elif threshold == "DoubleEle33UnseededLeg":
                suffix = "CaloIdL_MW\nUnseeded leg"
            elif threshold == "Ele23Ele12Leg1":
                suffix = "CaloIdL_TrackIdL_IsoVL Leg1"
            elif threshold == "Ele23Ele12Leg2":
                suffix = "CaloIdL_TrackIdL_IsoVL Leg2"
            else:
                raise ValueError("Couldn't find proper trigger name")

            plateau_cut = plateau_cuts[threshold]

            filename = threshold
            threshold = threshold.replace("Leg1", "").replace("Leg2", "").replace("SeededLeg", "").replace("UnseededLeg", "")

            plottype = "pt_high_threshold" if threshold == "Ele115" or threshold == "Ele135" else "pt_low_threshold"
            title = f"HLT_{threshold}_{suffix}"
            lumi = []
            for r in run:
                try:
                    l = lumis[r]
                except KeyError:
                    if r == "2022":
                        l = lumis["2022C"] + lumis["2022D"] + lumis["2022E"] + lumis["2022F"] + lumis["2022G"]
                    elif r == "2023":
                        l = lumis["2023B"] + lumis["2023C"] + lumis["2023D"]
                    else:
                        l = "X"
                if not isinstance(l, str):
                    l = round(l, 1)
                lumi.append(l)

            year = []
            for r in run:
                if "2022" in r:
                    year.append("2022")
                elif "2023" in r:
                    year.append("2023")
                else:
                    year.append("2024")

            rlabel = "2024"

            if mc_dataset == "events_2024D":
                eff1_kwargs = {"color": "#5790fc"}  # Blue
                effratio_kwargs = {"color": "#5790fc"}
                print("I am 2024D and I am using color Blue")
            elif mc_dataset == "events_2024E":
                eff1_kwargs = {"color": "#e42536"} # Red
                effratio_kwargs = {"color": "#e42536"}
                print("I am 2024E and I am using color Red")
            elif mc_dataset == "events_2024F":
                eff1_kwargs = {"color": "#49a843"} # Green
                effratio_kwargs = {"color": "#49a843"}  
                print("I am 2024F and I am using color Green")
            elif mc_dataset == "events_2024G":
                eff1_kwargs = {"color": "#5431f1"}  # Violet
                effratio_kwargs = {"color": "#5431f1"}
                print("I am 2024G and I am using color Violet")
            elif mc_dataset == "events_2024H":
                eff1_kwargs = {"color": "#ff33e9"} # Magenta
                effratio_kwargs = {"color": "#ff33e9"}
                print("I am 2024H and I am using color magenta")
            elif mc_dataset == "events_2024I":
                eff1_kwargs = {"color": "#f18e31"} # Orange
                effratio_kwargs = {"color": "#f18e31"}  
                print("I am 2024I and I am using color Orange")
            else:
                eff1_kwargs = {"color": "#7a21dd"} # Purple
                effratio_kwargs = {"color": "#7a21dd"} # Purple  
                print("I am else and I am using color purple")
            (
                hpt_barrel_pass1,
                hpt_barrel_all1,
                hpt_endcap_loweta_pass1,
                hpt_endcap_loweta_all1,
                hpt_endcap_higheta_pass1,
                hpt_endcap_higheta_all1,
                hpt_combined_pass1,
                hpt_combined_all1,
                heta_entire_pass1,
                heta_entire_all1,
                hphi_entire_pass1,
                hphi_entire_all1,
            ) = get_histograms(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{tocompare[0]}/{filename}_hists.root")

            (
                hpt_barrel_pass2,
                hpt_barrel_all2,
                hpt_endcap_loweta_pass2,
                hpt_endcap_loweta_all2,
                hpt_endcap_higheta_pass2,
                hpt_endcap_higheta_all2,
                hpt_combined_pass2,
                hpt_combined_all2,           
                heta_entire_pass2,
                heta_entire_all2,
                hphi_entire_pass2,
                hphi_entire_all2,
            ) = get_histograms(f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{tocompare[1]}/{filename}_hists.root")
            
            if _fig is None:
                skip_reference_efficieny=None
            else:
                skip_reference_efficieny=True

#            _fig = plot_ratio(
#                heta_entire_pass1,
#                heta_entire_all1,
#                heta_entire_pass2,
#                heta_entire_all2,
#                label1=f"{run[0]}",
#                label2=f"{run[1]}",
#                denominator_type="failing",
#                plottype="eta",
#                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_eta.png",
#                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_eta.pdf",
#                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 2.50$\nProbe electron $P_T> {plateau_cut}$ GeV"},
#                cms_kwargs={"loc": 1, "rlabel": rlabel},
#                eff1_kwargs=eff1_kwargs,
#                effratio_kwargs=effratio_kwargs,
#                efficiency_label="L1T + HLT Efficiency",
#                _fig=_fig,
#                _skip_fig1=skip_reference_efficieny    

#            _fig = plot_ratio(
#                hphi_entire_pass1,
#                hphi_entire_all1,
#                hphi_entire_pass2,
#                hphi_entire_all2,
#                label1=f"{run[0]}",
#                label2=f"{run[1]}",
#                denominator_type="failing",
#                plottype="phi",
#                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_phi.png",
#                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_phi.pdf",
#                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 2.50$\nProbe electron $P_T> {plateau_cut}$ GeV"},
#                cms_kwargs={"loc": 1, "rlabel": rlabel},
#                eff1_kwargs=eff1_kwargs,
#                effratio_kwargs=effratio_kwargs,
#                efficiency_label="L1T + HLT Efficiency",
#                _fig=_fig,
#                _skip_fig1=skip_reference_efficieny                   

            _fig = plot_ratio(
                hpt_combined_pass1,
                hpt_combined_all1,
                hpt_combined_pass2,
                hpt_combined_all2,
                label1=f"{run[0]}",
                label2=f"{run[1]}",
                denominator_type="failing",
                plottype=plottype,
                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_combined_pt.png",
                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_combined_pt.pdf",
                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 1.44$ or $1.57 < |\eta| < 2.50$"},
                cms_kwargs={"loc": 1, "rlabel": rlabel},
                eff1_kwargs=eff1_kwargs,
                effratio_kwargs=effratio_kwargs,
                efficiency_label="L1T + HLT Efficiency",
                _fig=_fig,
                _skip_fig1=skip_reference_efficieny 

#            _fig = plot_ratio(
#                hpt_barrel_pass1,
#                hpt_barrel_all1,
#                hpt_barrel_pass2,
#                hpt_barrel_all2,
#                label1=f"{run[0]}",
#                label2=f"{run[1]}",
#                denominator_type="failing",
#                plottype=plottype,
#                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_barrel_pt.png",
#                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_barrel_pt.pdf",
#                legend_kwargs={"title": f"{title}\n$0.00 < |\eta| < 1.44$"},
#                cms_kwargs={"loc": 1, "rlabel": rlabel},
#                eff1_kwargs=eff1_kwargs,
#                effratio_kwargs=effratio_kwargs,
#                efficiency_label="L1T + HLT Efficiency",
#                _fig=_fig,
#                _skip_fig1=skip_reference_efficieny 

#            _fig = plot_ratio(
#                hpt_endcap_loweta_pass1,
#                hpt_endcap_loweta_all1,
#                hpt_endcap_loweta_pass2,
#                hpt_endcap_loweta_all2,
#                label1=f"{run[0]}",
#                label2=f"{run[1]}",
#                denominator_type="failing",
#                plottype=plottype,
#                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_loweta_pt.png",
#                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_loweta_pt.pdf",
#                legend_kwargs={"title": f"{title}\n$1.57 < |\eta| < 2.00$"},
#                cms_kwargs={"loc": 1, "rlabel": rlabel},
#                eff1_kwargs=eff1_kwargs,
#                effratio_kwargs=effratio_kwargs,
#                efficiency_label="L1T + HLT Efficiency",
#                _fig=_fig,
#                _skip_fig1=skip_reference_efficieny 

#            _fig = plot_ratio(
#                hpt_endcap_higheta_pass1,
#                hpt_endcap_higheta_all1,
#                hpt_endcap_higheta_pass2,
#                hpt_endcap_higheta_all2,
#                label1=f"{run[0]}",
#                label2=f"{run[1]}",
#                denominator_type="failing",
#                plottype=plottype,
#                figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_higheta_pt.png",
#                #figure_path=f"/afs/cern.ch/user/s/ssaumya/Egamma/egamma-tnp/Full2024/{filename}_HLT_eff_higheta_pt.pdf",
#                legend_kwargs={"title": f"{title}\n$2.00 < |\eta| < 2.50$"},
#                cms_kwargs={"loc": 1, "rlabel": rlabel},
#                eff1_kwargs=eff1_kwargs,
#                effratio_kwargs=effratio_kwargs,
#                efficiency_label="L1T + HLT Efficiency",
#                _fig=_fig,
#                _skip_fig1=skip_reference_efficieny 

)

I am 2024D and I am using color Blue
I am 2024E and I am using color Red
I am 2024F and I am using color Green
I am 2024G and I am using color Violet
I am 2024H and I am using color magenta
I am 2024I and I am using color Orange
I am 2024D and I am using color Blue
I am 2024E and I am using color Red
I am 2024F and I am using color Green
I am 2024G and I am using color Violet
I am 2024H and I am using color magenta
I am 2024I and I am using color Orange
I am 2024D and I am using color Blue
I am 2024E and I am using color Red
I am 2024F and I am using color Green
I am 2024G and I am using color Violet
I am 2024H and I am using color magenta
I am 2024I and I am using color Orange
